In [ ]:
./checkpoints/brp-nas/Modelbrp-nas_datasetnasbench201_dstdatasetdesktop-cpu-core-i7-7820x-fp32_spliterratio5:4:91_dmodel64_round_0.pt
./checkpoints/brp-nas/Modelbrp-nas_datasetnasbench201_dstdatasetdesktop-cpu-core-i7-7820x-fp32_spliterratio5:4:91_dmodel64_round_0.pt

In [4]:
import torch

# 设置模型文件路径
model_path = './checkpoints/brp-nas/Modelbrp-nas_datasetnasbench201_dstdatasetdesktop-cpu-core-i7-7820x-fp32_spliterratio5:4:91_dmodel64_round_0.pt'

# 加载模型的权重
state_dict = torch.load(model_path)

# 逐层打印每个权重的形状
for name, param in state_dict.items():
    print(f"{name.ljust(60)} | Shape: {str(param.shape).ljust(30)}")


total_ops                                                    | Shape: torch.Size([1])               
total_params                                                 | Shape: torch.Size([1])               
model.total_ops                                              | Shape: torch.Size([1])               
model.total_params                                           | Shape: torch.Size([1])               
model.GCN.total_ops                                          | Shape: torch.Size([1])               
model.GCN.total_params                                       | Shape: torch.Size([1])               
model.GCN.gcn.total_ops                                      | Shape: torch.Size([1])               
model.GCN.gcn.total_params                                   | Shape: torch.Size([1])               
model.GCN.gcn.0.weight                                       | Shape: torch.Size([6, 600])          
model.GCN.gcn.0.bias                                         | Shape: torch.Size([600])    

/tmp/ipykernel_1076052/2332740234.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


In [ ]:
loss_rank = lossRank(logits, gt) * config.lambda_diff

In [ ]:
source_pre, auged_pre = torch.split(pre, pre.shape[0]//2, dim=0)
                loss_con = lossConsistency(source_pre, auged_pre) * config.lambda_consistency

In [ ]:
lossRank = DiffLoss(args.rankloss_type)
    lossConsistency = torch.nn.L1Loss()

In [2]:
import torch
import torch.nn as nn
import random

class CalPairDiff(nn.Module):
    def __init__(self, type):
        super(CalPairDiff, self).__init__()
        if type.lower() == 'l1':
            self.cal_loss = nn.L1Loss()
        elif type.lower() == 'l2':
            self.cal_loss = nn.MSELoss()
        elif type.lower() == 'kldiv':
            self.cal_loss = nn.KLDivLoss()
        
    def forward(self, predicts, target):
        B = predicts.shape[0]
        ori_pre = predicts
        ori_tar = target
        index = list(range(B))
        random.shuffle(index)
        predicts = predicts[index]
        target = target[index]
        v1 = ori_pre - predicts
        v2 = ori_tar - target
        loss = self.cal_loss(v1, v2)
        return loss

class DiffLoss(nn.Module):
    def __init__(self, type):
        super(DiffLoss, self).__init__()
        self.crit = CalPairDiff(type)

    def forward(self, predicts, target):
        loss = self.crit(predicts, target)
        return loss

loss = DiffLoss('l1')
a1=torch.tensor([0, 2, 4, 6, 8, 10, 12, 14, 16, 18], dtype=float)
a2=torch.tensor([1, 3, 5, 7, 9, 11, 13, 15, 17, 19], dtype=float)
l1 = loss(a1, a2)

b1=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=float)
b2=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=float)
l2 = loss(b1, b2)
